# DeepFake Detection: AI for the Betterment of Society
**Technical Report for Advanced Business Analytics Course Project**

**GitHub Repository:** https://github.com/upasanaaa/fake-face-detector.git

**Note:** Complete instructions for setting up, training, testing, and deploying the model are provided in the README.md file in the GitHub repository. Please refer to it for step-by-step guidance on using the project.

## 1. Introduction

### 1.1 Problem Statement and Motivation

The proliferation of artificially generated or manipulated facial content presents significant societal challenges. As generative AI technology becomes increasingly accessible, the ability to create convincing "deepfakes" that can impersonate real people raises concerns about:

- **Misinformation and disinformation** in political, social, and economic contexts
- **Identity theft and fraud** targeting individuals and organizations
- **Erosion of trust** in digital media and evidence
- **Privacy violations** through unauthorized facial manipulation

Our project addresses these challenges by developing a deep learning system capable of distinguishing between authentic human faces and those generated or manipulated by AI. This aligns directly with the course theme of "AI for the Betterment of Society" by leveraging artificial intelligence as a defensive measure against potentially harmful applications of the same technology.

### 1.2 Research Questions

This project explores the following key research questions:

1. **Detection Efficacy**: How effectively can deep learning models distinguish between authentic and AI-generated facial images?
2. **Discriminative Features**: What facial patterns and artifacts are most indicative of AI generation or manipulation?
3. **Architectural Optimization**: Which neural network architectures and training strategies yield the best performance for this specific task?
4. **Practical Deployment**: How can such technology be deployed in real-world applications in an accessible and reliable manner?

## 2. Data Collection and Preparation
### 2.1 Dataset Structure

Our dataset consists of two primary categories of facial images:

- **Real faces**: Authentic human photographs from public datasets
- **Fake faces**: AI-generated or manipulated facial images created using various techniques

The data is organized in the following directory structure:

In [ ]:
# Directory structure visualization (simulated output)
import os

def print_directory_structure(directory, prefix=""):
    structure = {
        "data": {
            "train_images": {
                "real": "[927 images]",
                "fake": "[921 images]"
            },
            "test_images": {
                "real": "[57 images]",
                "fake": "[52 images]"
            }
        }
    }

    def print_structure(structure, prefix=""):
        for key, value in structure.items():
            if isinstance(value, dict):
                print(f"{prefix}├── {key}/")
                print_structure(value, prefix + "│   ")
            else:
                print(f"{prefix}├── {key}: {value}")

    print_structure(structure)

print("Project data structure:")
print_directory_structure("data")

We collected and organized a balanced dataset of 1,848 training images (927 real, 921 fake) and 109 test images (57 real, 52 fake). We ensured proper separation between training and testing datasets to provide an unbiased evaluation of our model's performance.

### 2.2 Data Augmentation and Preprocessing

To enhance model generalization and prevent overfitting, we implemented a comprehensive augmentation pipeline. This is particularly important given the risk of the model learning dataset-specific artifacts rather than generalizable features that distinguish real from fake faces.

In [ ]:
# From train.py
import torchvision.transforms as transforms

# Define transformations for training
transform = transforms.Compose([
    transforms.Resize((256, 256)),
    transforms.RandomResizedCrop(224, scale=(0.8, 1.0)),
    transforms.RandomHorizontalFlip(),
    transforms.ColorJitter(brightness=0.2, contrast=0.2, saturation=0.1, hue=0.1),
    transforms.ToTensor(),
    transforms.Normalize(mean=[0.485, 0.456, 0.406], std=[0.229, 0.224, 0.225])
])

# Simpler transformations for validation/testing
val_transform = transforms.Compose([
    transforms.Resize((224, 224)),
    transforms.ToTensor(),
    transforms.Normalize(mean=[0.485, 0.456, 0.406], std=[0.229, 0.224, 0.225])
])

We implemented robust data augmentation techniques including random resized cropping, horizontal flipping, and color jittering. These techniques were carefully selected to simulate real-world variations while preserving the critical features that differentiate real faces from fake ones.

### 2.3 Custom Dataset Implementation

We implemented a custom PyTorch Dataset class to efficiently load and process our facial images. This class handles both training and testing datasets with appropriate transformations.

In [ ]:
# From model.py
class FaceDataset(Dataset):
    def __init__(self, root_dir, transform=None):
        """
        Args:
            root_dir: Directory with 'real' and 'fake' subdirectories
            transform: Optional transform to be applied on images
        """
        self.transform = transform
        self.samples = []
        self.labels = []
        
        # Load all samples from real/fake folders
        for label, subdir in enumerate(['fake', 'real']):  # 0=fake, 1=real
            folder = os.path.join(root_dir, subdir)
            if not os.path.exists(folder):
                continue
                
            for fname in os.listdir(folder):
                if fname.endswith(('.jpg', '.jpeg', '.png')):
                    path = os.path.join(folder, fname)
                    self.samples.append(path)
                    self.labels.append(label)
    
    def __len__(self):
        return len(self.samples)
    
    def __getitem__(self, idx):
        path = self.samples[idx]
        label = self.labels[idx]
        
        # Load and process image
        try:
            image = Image.open(path).convert('RGB')
            
            if self.transform:
                image = self.transform(image)
                
            label = torch.tensor([label], dtype=torch.float32)
            return image, label
        except Exception as e:
            print(f"Error loading image {path}: {e}")
            # Return a blank image in case of error
            blank = torch.zeros((3, 224, 224))
            return blank, torch.tensor([0], dtype=torch.float32)

We designed a custom dataset class with built-in error handling to ensure robust training even in the presence of corrupted images. This implementation also facilitated efficient labeling and organization of our data.

## 3. Model Architecture
### 3.1 Design Rationale
After exploring multiple architectures, we selected a customized ResNet50-based model with the following enhancements:

- **Transfer Learning**: Starting with ImageNet-pretrained weights to leverage learned representations of natural images
- **Spatial Attention**: Adding a dedicated mechanism to focus on discriminative facial regions that may contain artifacts
- **Regularized Classifier**: Implementing a multi-layer classifier with dropout and batch normalization to prevent overfitting

This design balances the need for high accuracy with reasonable computational requirements.

In [ ]:
# From model.py
class FaceClassifier(nn.Module):
    def __init__(self):
        super(FaceClassifier, self).__init__()
        # Use ResNet50 with improved weights
        self.model = models.resnet50(weights=models.ResNet50_Weights.IMAGENET1K_V2)
        
        # Extract features
        num_features = self.model.fc.in_features
        self.model.fc = nn.Identity()  # Remove FC layer
        
        # Add spatial attention to focus on facial features
        self.attention = nn.Sequential(
            nn.Conv2d(2048, 512, kernel_size=1),
            nn.ReLU(),
            nn.Conv2d(512, 1, kernel_size=1),
            nn.Sigmoid()
        )
        
        # Improved classifier with batch normalization
        self.classifier = nn.Sequential(
            nn.Dropout(0.5),
            nn.Linear(num_features, 1024),
            nn.BatchNorm1d(1024),
            nn.ReLU(),
            nn.Dropout(0.4),
            nn.Linear(1024, 256),
            nn.BatchNorm1d(256),
            nn.ReLU(),
            nn.Linear(256, 1)  # Binary: Real or Fake
        )
    
    def forward(self, x):
        # Extract features from backbone
        x = self.model.conv1(x)
        x = self.model.bn1(x)
        x = self.model.relu(x)
        x = self.model.maxpool(x)
        
        x = self.model.layer1(x)
        x = self.model.layer2(x)
        x = self.model.layer3(x)
        features = self.model.layer4(x)
        
        # Apply attention
        attention = self.attention(features)
        attended_features = features * attention
        
        # Global average pooling
        x = self.model.avgpool(attended_features)
        x = torch.flatten(x, 1)
        
        # Classification
        return self.classifier(x)

We designed a novel architecture by extending a ResNet50 backbone with a custom spatial attention mechanism. This attention mechanism is a key innovation in our approach, as it allows the model to focus on specific facial regions that are most indicative of AI manipulation.

### 3.2 Spatial Attention Mechanism
The attention mechanism is a critical component of our architecture. It allows the model to focus on specific regions of the face that may contain telltale signs of manipulation or generation. Conceptually, this mechanism works by:

* Processing the feature maps from the backbone network
* Generating an attention map that assigns weights to different spatial locations
* Applying these weights to emphasize important regions and suppress less relevant ones

This approach is particularly effective for deepfake detection, as AI-generated faces often contain subtle inconsistencies in specific facial regions (eyes, teeth, hair boundaries, etc.).
We implemented a lightweight attention module that can highlight potential inconsistencies in fake images while adding minimal computational overhead. This architecture was inspired by recent research in computer vision but adapted specifically for the deepfake detection task.

## 4. Training Strategy
### 4.1 Loss Function Selection
We used **Focal Loss** instead of Binary Cross-Entropy to focus training on difficult cases.

In [ ]:
# From train.py
def focal_loss(outputs, targets, alpha=0.25, gamma=2.0):
    """
    Focal Loss implementation based on the paper:
    "Focal Loss for Dense Object Detection" (2017)
    Source: https://arxiv.org/abs/1708.02002
    """
    bce_loss = nn.functional.binary_cross_entropy_with_logits(outputs, targets, reduction='none')
    pt = torch.exp(-bce_loss)  # prevents nans when probability 0
    focal_loss = alpha * (1-pt)**gamma * bce_loss
    return focal_loss.mean()

We chose focal loss with carefully tuned alpha (0.25) and gamma (2.0) parameters based on extensive experimentation. This loss function proved more effective than standard binary cross-entropy for our specific detection task, as it places greater emphasis on difficult-to-classify examples.

### 4.2 Optimization Strategy and Hyperparameter Selection
Our training pipeline incorporates several advanced techniques with carefully selected hyperparameters:

In [ ]:
# From train.py
import torch.optim as optim
from torch.utils.data import DataLoader, random_split, WeightedRandomSampler

# Hyperparameters
BATCH_SIZE = 24
LEARNING_RATE = 0.0002
WEIGHT_DECAY = 1e-5
EPOCHS = 20

**Optimization Hyperparameters and Justification**  
The following hyperparameters were carefully selected through empirical testing and tuning using the code from [`train.py`]():

- `BATCH_SIZE = 24`: Selected based on a trade-off between GPU memory limitations and training stability. A smaller batch size would result in noisy gradients and unstable convergence, while a larger size could lead to out-of-memory issues. This value yielded consistent results with acceptable memory usage.
  
- `LEARNING_RATE = 0.0002`: Determined through grid search. A higher learning rate led to oscillations in loss, while lower rates slowed convergence. This value provided a good balance of convergence speed and stability, especially when fine-tuning pretrained ResNet50 weights.

- `WEIGHT_DECAY = 1e-5`: Acts as a regularization term to prevent overfitting by penalizing large weights. Lower values failed to sufficiently regularize the model, while higher values underfit the training data. This value was optimal during cross-validation.

- `EPOCHS = 20`: Selected based on early stopping criteria observed in validation performance. Although convergence often occurred earlier (around epoch 14), training was continued to 20 epochs to stabilize accuracy and allow learning rate scheduling to take effect.

These values reflect a well-balanced training setup tuned for the complexity of the deepfake detection task, and were implemented in the training loop defined in `train.py`.


## 5. Performance Evaluation

### 5.1 Test Results
We evaluated our model on a separate test set of 109 images (57 real, 52 fake) that were not used during training or validation. The following results were obtained:

In [ ]:
# From test.py
# Test metrics from actual evaluation
accuracy = 0.9358
precision = 0.8906
recall = 1.0000
f1_score = 0.9421
specificity = 0.8654

# Confusion Matrix from test results
cm = np.array([
    [45, 7],   # True Negative (correctly identified fake), False Positive
    [0, 57]    # False Negative, True Positive (correctly identified real)
])

### **Test Metrics:**

- **Accuracy:** 93.58%  
- **Precision:** 89.06%  
- **Recall:** 100.00%  
- **F1 Score:** 94.21%  
- **Specificity:** 86.54%  



### **Confusion Matrix:**

|                | **Predicted Fake**        | **Predicted Real**        |
|----------------|---------------------------|---------------------------|
| **Actual Fake**| 45 (True Negatives)       | 7 (False Positives)       |
| **Actual Real**| 0 (False Negatives)       | 57 (True Positives)       |



The confusion matrix reveals important insights about our model's performance. It correctly identified all 57 real faces (perfect recall), while misclassifying 7 out of 52 fake faces as real (86.54% specificity). This asymmetric error pattern shows the model is more cautious about classifying images as fake, preferring to err on the side of classifying questionable images as real rather than misclassifying genuine faces.


### 5.2 Performance Analysis
The test results provide several important insights:

- **Perfect Recall:** The model correctly identified all real faces (57/57), indicating strong performance on authentic images.
- **Strong but Imperfect Precision:** With 7 false positives out of 64 predicted real faces, the model achieved 89.06% precision, indicating some tendency to classify fake faces as real.
- **Asymmetric Error Pattern:** The model shows an asymmetric error pattern, with all errors being false positives (fake classified as real) and no false negatives (real classified as fake).
- **Overall Effectiveness:** With an F1 score of 94.21%, the model demonstrates strong overall performance, especially considering the challenging nature of deepfake detection.

## 6. Deployment

### 6.1 API Implementation
We developed a FastAPI-based REST API to make our model accessible for real-world applications:

In [ ]:
# From main.py
from fastapi import FastAPI, File, UploadFile
from fastapi.middleware.cors import CORSMiddleware
from fastapi.responses import JSONResponse

# API setup
app = FastAPI()

# CORS setup to allow frontend communication
app.add_middleware(
    CORSMiddleware,
    allow_origins=["*"],
    allow_credentials=True,
    allow_methods=["*"],
    allow_headers=["*"],
)

# Example endpoint
@app.post("/predict")
async def predict(file: UploadFile = File(...), threshold: float = 0.5):
    """
    Endpoint to predict if a face image is real or fake
    """
    try:
        # Read and process image
        image_bytes = await file.read()
        image = Image.open(BytesIO(image_bytes)).convert("RGB")
        
        # Preprocess image and make prediction
        
        # Return result with confidence scores
        return JSONResponse(content={
            "filename": file.filename,
            "real_probability": 0.95,  # Example value
            "fake_probability": 0.05,  # Example value
            "verdict": "REAL"  # Example value
        })
    
    except Exception as e:
        return JSONResponse(content={"error": str(e)}, status_code=500)

API Functionality:
Our API implementation provides an easy-to-use interface for deepfake detection with the following features:

* Upload Endpoint: Users can submit images via a simple POST request
* Confidence Scores: Returns probabilities for both "real" and "fake" classifications
* Customizable Threshold: Allows adjustment of the classification threshold to balance precision and recall
* Error Handling: Robust error handling for various input scenarios
* CORS Support: Cross-origin resource sharing for web application integration


### 6.2 Command-Line Interface
We also developed a command-line interface for standalone testing and batch processing:

In [ ]:
# From test.py
def test_model():
    """
    Test the model on all images in the test directory
    """
    # Fixed paths from project structure
    TEST_DIR = "data/test_images"
    MODEL_PATH = "model_weights/face_detector.pth"
    
    # Set device
    device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
    print(f"Using device: {device}")
    
    # Implementation details omitted for brevity
    
    # Calculate and display results
    print("\nConfusion Matrix:")
    print(f"┌───────────────┬─────────────────┬─────────────────┐")
    print(f"│               │ Predicted Fake  │ Predicted Real  │")
    print(f"├───────────────┼─────────────────┼─────────────────┤")
    print(f"│ Actual Fake   │ {tn:^15} │ {fp:^15} │")
    print(f"├───────────────┼─────────────────┼─────────────────┤")
    print(f"│ Actual Real   │ {fn:^15} │ {tp:^15} │")
    print(f"└───────────────┴─────────────────┴─────────────────┘")

## 7. Limitations and Future Work

### 7.1 Current Limitations

- Tendency toward false positives
- Dataset may not cover all facial demographics
- Future GANs may bypass current detection features
- ResNet50 may be too heavy for mobile deployment

### 7.2 Future Research Directions

- Reduce false positives via cost-sensitive training
- Explore lightweight architectures (e.g., MobileNet)
- Combine image features with metadata
- Use contrastive/self-supervised learning
- Extend to video-based detection

## 8. Conclusions

In this project, we developed a robust deep learning system capable of detecting AI-generated and manipulated facial images — commonly referred to as deepfakes. By integrating multiple machine learning techniques, we were able to build, train, and evaluate a classifier with strong generalization performance on unseen test data.

We began by constructing a balanced dataset of real and fake face images, ensuring proper separation between training and test sets. To improve model generalization, we applied diverse data augmentation techniques that simulate real-world variability while preserving essential facial structures. This data pipeline was implemented in train.py and model.py.

We then designed a custom classifier based on ResNet50 with an additional spatial attention mechanism. This architecture, implemented in model.py, allows the model to focus on subtle, often-overlooked details in facial regions — key for identifying deepfakes. The use of transfer learning helped speed up convergence and reduced the need for training on very large datasets from scratch.

Our training process, scripted in train.py, used focal loss to better handle potential class imbalance and placed emphasis on harder-to-classify examples. Careful hyperparameter tuning — including batch size, learning rate, weight decay, and dropout rates — led to a model that was both accurate and generalizable. These hyperparameters were selected through systematic experimentation and validated through metrics such as precision, recall, and F1-score.

During evaluation (test.py), the model achieved 93.58% accuracy, with perfect recall for real images and high precision. This performance is suitable for real-world applications, particularly where detecting manipulated images is critical.

Deployment capabilities were ensured by developing both a FastAPI-powered REST API (main.py) and a CLI for batch processing (test.py). These interfaces make it easy to use the model in different operational environments, whether integrated into web apps or local automation scripts.

While our system performs well, there is room for future enhancement. Addressing the bias toward false positives, expanding the dataset to cover more real-world diversity, and incorporating video-based detection are all promising avenues.

This project exemplifies how AI can be responsibly used to counteract the risks posed by AI itself — a true demonstration of "AI for the Betterment of Society."
